# Выбор региона для разработки новых нефтяных скважин

Добывающей компании «ГлавРосГосНефть» необходимо решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нам предстоит построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой Bootstrap.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

###### Данное исследование разделим на несколько этапов.

##### 1. Загрузка и подготовка данных:
* [1.1. Загрузка библиотек, изучение файлов с данными, получение общей информации.](#section1)
* [1.2. Изучение коэффициентов корреляции.](#section2)

##### 2. Обучение и проверка модели:
* [2.1. Разбиение данных на выборки.](#section3)
* [2.2. Обучение и предсказания модели на валидационной выборке.](#section4)
* [2.3. Получение среднего запаса предсказанного сырья, r2 и RMSE.](#section5)

##### 3. Подготовка к расчёту прибыли:
* [3.1. Сохранение значений для расчётов в отдельные переменные.](#section6)
* [3.2. Рассчет достаточного объема сырья для безубыточной разработки новой скважины.](#section7)

##### 4. Расчёт прибыли и рисков :
* [4.1. Написание функции для расчёта прибыли по выбранным скважинам и предсказаниям модели.](#section8)
* [4.2. Нахождение средней прибыли, 95%-ого доверительного интервала и риска убытков.](#section9)

##### 5. Общий вывод:
* [5.1. Формулировка общего вывода.](#section10)

<a id='section1'></a>

# 1. Загрузка и подготовка данных

## 1.1. Загрузка библиотек, изучение файлов с данными, получение общей информации.

### Импортируем все необходимые библиотеки.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from sklearn.linear_model import LinearRegression


#Импортируем необходимые метрики 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score,r2_score
from sklearn.metrics import confusion_matrix, roc_curve, mean_squared_error

from sklearn.utils import shuffle

import seaborn as sns

from math import sqrt
from numpy.random import RandomState



### Прочитаем файл с данными.

In [2]:
#<чтение данных из файла>
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')

### Получим первые строки таблицы.

In [3]:
print(df1.head())
print(df2.head())
print(df3.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


Известно, что в каждой таблице присутствуют следующие столбцы:

признаки:
* id — уникальный идентификатор скважины
* f0, f1, f2 — три признака точек

целевой признак:
* product — объём запасов в скважине (тыс. баррелей).

### Изучим общую информацию таблицы.

In [4]:
print(df1.info())
print(df2.info())
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

*Рассмотрим полученную информацию подробнее. Всего в таблицах по 5 столбцов и 100000 строк, присутствуют два типа данных. Количество значений в столбцах не различается. Это говорит о том, что в данных нет пропущенных значений.*

In [5]:
print(df1.describe())
print(df2.describe())
print(df3.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

*Рассмотрим полученную информацию подробнее. Аномалий не обнаружено. Можно заметить, что в каждом регионе есть точки, где объем запасов в скважине нулевые. Наибольший средний объем запаса нефти наблюдается у 3-его региона, а наименьший во 2-ом регионе.*

<a id='section2'></a>

## 1.2. Изучение коэффициентов корреляции.

In [6]:
print(df1.corr())
print(df2.corr())
print(df3.corr())

               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000


*Можно заметить, что во 2-ом регионе признак f2 сильно корелирует с product.*

## ВЫВОД 
На данном этапе проекта мы изучили данные о каждом из регионов. Проверили данные на наличие пропусков.
Обнаружили, что в каждом регионе есть пустые скважины. Наибольший средний объем запасов нефти в скважине зафиксирован в 3-ем регионе, а наименьший — во 2-ом. Так же обнаружили корреляцию между признаками `f2` и `product` во 2-ом регионе, в других регионах эти признаки не коррелируют.

<a id='section3'></a>

# 2. Обучение и проверка модели.

## 2.1. Разбиение данных на выборки.

### Разделим наш набор данных на следующие выборки:
* 75% - обучающий набор,
* 25% - валидационный набор.

Удалим признак `id`, он не представляет ценности для качества будущей модели.

In [7]:
state=np.random.RandomState(12345)

#1-ый регион
feature_1 = df1.drop(['id', 'product'], axis=1)
target_1 = df1['product']
feature_train_1, feature_valid_1, target_train_1, target_valid_1 = train_test_split(
        feature_1, target_1, test_size=0.25, random_state=state)

#2-ой регион
feature_2 = df2.drop(['id', 'product'], axis=1)
target_2 = df2['product']
feature_train_2, feature_valid_2, target_train_2, target_valid_2 = train_test_split(
        feature_2, target_2, test_size=0.25, random_state=state)

#3-ий регион
feature_3 = df3.drop(['id', 'product'], axis=1)
target_3 = df3['product']
feature_train_3, feature_valid_3, target_train_3, target_valid_3 = train_test_split(
        feature_3, target_3, test_size=0.25, random_state=state)

<a id='section4'></a>

## 2.2. Обучение и предсказания моделей на валидационной выборке.

In [8]:
def pred_model(feature_train, target_train, feature_valid):
    model = LinearRegression()
    model.fit(feature_train, target_train)
    predictions = model.predict(feature_valid)
    return predictions

<a id='section5'></a>

In [9]:
#1-ый регион
predictions_1 = pred_model(feature_train_1, target_train_1, feature_valid_1)

#2-ой регион
predictions_2 = pred_model(feature_train_2, target_train_2, feature_valid_2)

#3-ий регион
predictions_3 = pred_model(feature_train_3, target_train_3, feature_valid_3)

## 2.3. Получение среднего запаса предсказанного сырья, r2 и RMSE.

In [10]:
#1-ый регион
RMSE_1 = sqrt(mean_squared_error(target_valid_1, predictions_1)) #квадратный корень из средней квадратичной ошибки
mean_1 = predictions_1.mean() #средний запас предсказанного сырья
r2_1 = r2_score(target_valid_1, predictions_1) #функция оценки регрессии

#2-ой регион
RMSE_2 = sqrt(mean_squared_error(target_valid_2, predictions_2))
mean_2 = predictions_2.mean()
r2_2 = r2_score(target_valid_2, predictions_2)

#3-ий регион
RMSE_3 = sqrt(mean_squared_error(target_valid_3, predictions_3))
mean_3 = predictions_3.mean()
r2_3 = r2_score(target_valid_3, predictions_3)

In [11]:
result = pd.DataFrame(
    {'region': ['1-ый регион', '2-ой регион', '3-ий регион'],
     'RMSE': [RMSE_1, RMSE_2, RMSE_3],
    'mean': [mean_1, mean_2, mean_3],
    'r2': [r2_1, r2_2, r2_3]})
result

,region,RMSE,mean,r2
0,1-ый регион,37.579422,92.592568,0.279943
1,2-ой регион,0.889737,68.769951,0.999626
2,3-ий регион,39.958042,95.087528,0.202615


*Можно заметить, что модель предсказала средний запас нефти на валидационной выборке почти не отличающийся от фактических запасов нефти в каждом из регионов.
Чем меньше метрика RSME, тем лучше. Видим, что наименьший показатель RSME во 2-ом регионе.
Также, наблюдаем, что показатель метрики R2 во 2-ом регионе очень высокий, что нельзя сказать о 1-ом и 3-ем регионе, где модель отработала недостаточно точно.*

## ВЫВОД 
На данном этапе проекта, мы разделили исходные данные на 2 выборки: обучающую и валидационную.
Обучили модель линейной регрессии (LinearRegression) и получили предсказания. Также, проанализировали метрики RMSE, r2 и mean. И сделали вывод, что модель достаточно хорошо предсказывает объем нефти во 2-ом регионе, а на 1-ом и 3-ем регионе модель работает недостаточно хорошо.

<a id='section6'></a>

# 3. Подготовка к расчёту прибыли.

## 3.1. Сохранение значений для расчётов в отдельные переменные.

In [12]:
N_POINT = 500 #Кол-во точек исследуемых при разведке региона
N_BEST_POINT = 200 #Кол-во лучших точек для разработки
BUDGET = 10e9 #Бюджет на разработку скважин в регионе
UNIT_INCOME = 450000 #Доход с каждой единицы продукта
MAX_PROB_LOSS = 0.025 #максимальная вероятность убытков 

<a id='section7'></a>

## 3.2. Рассчет достаточного объема сырья для безубыточной разработки новой скважины.

### Бюджет на 1 скважину:

In [13]:
BUDGET_1_POINT = BUDGET / N_BEST_POINT
BUDGET_1_POINT

50000000.0

### Достаточный объем сырья для безубыточной разработки новой скважины:

In [14]:
BUDGET_1_POINT / UNIT_INCOME

111.11111111111111

*Объём сырья для безубыточной разработки новой скважины состовляет 111.11 единиц продукта. Этот объем гораздо больше, чем среднее предсказание объема нефти во всех регионах.*



## ВЫВОД 
На данном этапе проекта можно отметить, что для безубыточной разработки новой скважины необходимо переработать минимум 112 единиц продукта. Также выяснили, что предсказания среднего объема нефти во всех регионах гораздо меньше этого объема.

<a id='section8'></a>

# 4. Расчёт прибыли и рисков.

## 4.1. Написание функции для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [15]:
def revenue(target, probabilities, count):
    probs_best = probabilities.sort_values(ascending = False)
    income = target[probs_best.index][:count].sum() * UNIT_INCOME
    revenue_final = income - BUDGET
    return revenue_final

In [16]:
predictions_1 = pd.Series(predictions_1, index=target_valid_1.index)
predictions_2 = pd.Series(predictions_2, index=target_valid_2.index)
predictions_3 = pd.Series(predictions_3, index=target_valid_3.index)

<a id='section9'></a>

## 4.2. Нахождение средней прибыли, 95%-ого доверительного интервала и риска убытков.

### Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [17]:
def average_profit(target_valid, predictions, region):
    bootstrap = []
    values = []
    
    for i in range(1000):
        target_subsample = target_valid.sample(n = N_POINT, replace=True, random_state = state)
        probs_subsample = predictions[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, N_BEST_POINT))

    values = pd.Series(values)
    lower = values.quantile(0.025)/ 10e6
    upper = values.quantile(0.975)/ 10e6

    mean = values.mean()/ 10e6
    confidence_interval = (lower, upper)
    risk_of_loss = (values < 0).mean()* 100
    bootstrap.append({'region': region,
                            'lower': lower, 
                            'upper': upper,
                            'risk_of_loss': risk_of_loss,
                            'mean': mean, 
                            'confidence_interval': confidence_interval})
    return pd.DataFrame(bootstrap)

In [18]:
region_1 = '1-ый регион'
bootstrap_1 = average_profit(target_valid_1, predictions_1, region_1)

In [19]:
region_2 = '2-ой регион'
bootstrap_2 = average_profit(target_valid_2, predictions_2, region_2)

In [20]:
region_3 = '3-ий регион'
bootstrap_3 = average_profit(target_valid_3, predictions_3, region_3)

### Объединим в единую таблицу полученные результаты.

In [21]:
result = pd.concat([bootstrap_1, bootstrap_2, bootstrap_3])
result

,region,lower,upper,risk_of_loss,mean,confidence_interval
0,1-ый регион,-7.618781,95.784653,4.8,42.389724,"(-7.618781389036274, 95.78465319517828)"
0,2-ой регион,10.806690,92.857444,0.6,51.325670,"(10.80668952339621, 92.85744392324963)"
0,3-ий регион,-14.280063,89.338057,7.4,38.112036,"(-14.280063008786875, 89.33805657504003)"


*Самый высокий показатель средней прибыли во 2-ом регионе, самый низкий в 1-ом. Также, во 2-ом регионе риск убытков составляет 0.6%, что в 8 раз меньше чем в 1-ом. В 1-ом и в 3-ем регионе уровень риска превышает допустимый уровень в 2.5%.*


<a id='section10'></a>

# 5. Общий вывод.

Добывающей компании «ГлавРосГосНефть» необходимо было решить, где бурить новую скважину.
Нам были предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нам необходимо было построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Также, необходимо было проанализировать возможную прибыль и риски.
    
На первом этапе проекта мы изучили данные о каждом из регионов. Проверили данные на наличие пропусков. Обнаружили, что в каждом регионе есть пустые скважины. Наибольший средний объем запасов нефти в скважине зафиксирован в 3-ем регионе, а наименьший — во втором. Так же обнаружили корреляцию между признаками `f2` и `product` во 2-ом регионе, в других регионах эти признаки не коррелируют.
    
Следующим этапом нашего проекта стало - обучение и проверка модели. Мы разделили исходные данные на 2 выборки: обучающую и валидационную. Обучили модель линейной регрессии (LinearRegression) и получили предсказания. Также, проанализировали метрики RMSE, r2 и mean. И сделали вывод, что модель достаточно хорошо предсказывает объем нефти во 2-ом регионе, а на 1-ом и 3-ем регионе модель работает недостаточно хорошо.

Далее мы начали подготовку к расчету прибыли. Для этого мы сохранили значения для расчётов в отдельных переменных.  Посчитали  достаточный объем сырья для безубыточной разработки новой скважины и определили, что для безубыточной разработки новой скважины необходимо переработать минимум 112 единиц продукта. Также оказалось, что предсказания среднего объема нефти во всех регионах гораздо меньше этого объема.
    
Затем мы приступили непосредственно к  расчету прибыли и рисков. Мы написали функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели. Затем применили технику `Bootstrap` с 1000 выборок, чтобы найти распределение прибыли. И нашли среднюю прибыль, 95%-й доверительный интервал и риск убытков. Оказалось, что  самый высокий показатель средней прибыли во 2-ом регионе, самый низкий в 1-ом. Также выяснили,  что во 2-ом регионе риск убытков составляет 0.3%, что почти в 8 раз меньше чем в 1-ом. В 1-ом и в 3-ем регионе уровень риска превышает допустимый уровень в 2.5%.
    
Таким образом, полученные результаты позволяют отметить, что 2-ой регион наиболее перспективный. Добыча нефти в этом регионе приносит наименьший риск, и при этом, может принести наибольшую прибыль. 
Менее перспективным оказался 3-ий регион, т.к. добыча нефти в этом регионе связана с большим риском и наименьшей прибылью.